# CSV Builder

## Importo librerias necesarias

In [1]:
#Pandas
import pandas as pd
#Numpy
import numpy as np
#Datetime
import datetime as dt
#Pendulum
import pendulum

## Files necesarios

In [2]:
# Nota: Colocarlos en la carpeta src.
# valor_educacion.csv
# fiuba_1_postulantes_educacion.csv
# fiuba_1_postulantes_educacion_2.csv
# fiuba_1_postulantes_educacion_3.csv
# fiuba_2_postulantes_genero_y_edad.csv
# fiuba_2_postulantes_genero_y_edad_2.csv
# fiuba_2_postulantes_genero_y_edad_3.csv
# fiuba_3_vistas.csv
# fiuba_3_vistas_2.csv
# fiuba_3_vistas_3.csv
# fiuba_4_postulaciones.csv
# fiuba_4_postulaciones_2.csv
# fiuba_5_avisos_online.csv
# fiuba_5_avisos_online_2.csv
# fiuba_6_avisos_detalle.csv
# fiuba_6_avisos_detalle_2.csv
# fiuba_6_avisos_detalle_3.csv
# fiuba_6_avisos_detalle_4.csv

## Proceso a los postulantes

## Proceso informacion academica de los postulantes

In [3]:
#Informacion de nivel educativo de los postulantes
post_educacion = pd.concat([
    pd.read_csv('src/fiuba_1_postulantes_educacion.csv'), 
    pd.read_csv('src/fiuba_1_postulantes_educacion_2.csv'),
    pd.read_csv('src/fiuba_1_postulantes_educacion_3.csv')], axis=0)

In [4]:
#Mapeo de puntaje del nivel educativo
post_educacion = pd.merge(
    post_educacion,
    pd.read_csv('src/valor_educacion.csv', encoding='utf-8'),
    how='inner', left_on=['nombre','estado'], right_on = ['nombre','estado'])

In [5]:
# Busco la maxima educacion que tiene cada postulante.
max_post_educacion = post_educacion.groupby(['idpostulante'])['valor_educacion'].max().reset_index()
# Hago el join para filtrar todas las educaciones que no me interesan, dejando solo la maxima educacion.
post_educacion = pd.merge(post_educacion, max_post_educacion,  how='inner',left_on=['idpostulante','valor_educacion'],right_on = ['idpostulante','valor_educacion'])
post_educacion.drop(['nombre', 'estado'], inplace = True, axis=1)
post_educacion.drop_duplicates(inplace=True)

## Proceso informacion de genero y edad de los postulantes

In [6]:
#Informacion de genero y edad de los postulantes
post_gen_edad = pd.concat([
    pd.read_csv('src/fiuba_2_postulantes_genero_y_edad.csv'),
    pd.read_csv('src/fiuba_2_postulantes_genero_y_edad_2.csv'),
    pd.read_csv('src/fiuba_2_postulantes_genero_y_edad_3.csv')], axis=0)

In [7]:
post_gen_edad['sexo'] = post_gen_edad['sexo'].astype('category')

In [8]:
post_gen_edad.drop_duplicates(inplace=True)

In [9]:
post_gen_edad.loc[post_gen_edad['sexo'] == '0.0','sexo'] = np.nan
post_gen_edad.loc[post_gen_edad['sexo'] == 'NO_DECLARA','sexo'] = np.nan

cols = post_gen_edad.select_dtypes(['category']).columns
post_gen_edad[cols] = post_gen_edad[cols].apply(lambda x: x.cat.codes)

In [10]:
# Modifico el tipo de dato de la fecha.
post_gen_edad.fechanacimiento = post_gen_edad.fechanacimiento.apply(pd.to_datetime, errors='coerce')

now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)
post_gen_edad['edad'] = (now - pd.to_datetime(post_gen_edad['fechanacimiento'])).astype('<m8[Y]')
post_gen_edad.drop('fechanacimiento', inplace=True, axis=1)

In [11]:
post_gen_edad.drop_duplicates('idpostulante', inplace=True) 

In [12]:
post_gen_edad.fillna(0, inplace=True)

In [13]:
post_gen_edad['edad'] = post_gen_edad['edad'].astype(int)

In [14]:
post_gen_edad['sexo'] = post_gen_edad['sexo'].apply(lambda x: 0 if x == -1 else (-1 if x == 2 else 1))

In [15]:
#Mergeo la nueva informacion
postulantes = pd.merge(post_educacion, post_gen_edad, how='outer', on='idpostulante')

In [16]:
postulantes.fillna(-1, inplace=True)

## Proceso a las vistas

In [17]:
vistas = pd.concat([
    pd.read_csv('src/fiuba_3_vistas.csv'),
    pd.read_csv('src/fiuba_3_vistas_2.csv'),
    pd.read_csv('src/fiuba_3_vistas_3.csv')], axis=0)

In [18]:
vistas = vistas.drop_duplicates()
vistas = vistas.rename({'idAviso': 'idaviso'}, axis=1)

In [19]:
vistas['timestamp'] = vistas['timestamp'].apply(lambda x: pendulum.parse(x))

In [20]:
vistas['month']     = vistas['timestamp'].apply(lambda x: x.month).astype(int) 
vistas['day']       = vistas['timestamp'].apply(lambda x: x.day).astype(int) 
vistas['dayofweek'] = vistas['timestamp'].apply(lambda x: x.day_of_week).astype(int) 
vistas['hour']      = vistas['timestamp'].apply(lambda x: x.hour).astype(int) 

In [21]:
vistas = vistas.drop('timestamp', axis=1)

In [22]:
vistas = vistas.fillna(-1)

## Proceso mas postulaciones

In [23]:
postulaciones = pd.concat([
    pd.read_csv('src/fiuba_4_postulaciones.csv'),
    pd.read_csv('src/fiuba_4_postulaciones_2.csv')], axis=0)

In [24]:
postulaciones.drop_duplicates(inplace=True)

In [25]:
postulaciones.rename({'fechapostulacion': 'timestamp'}, axis=1, inplace=True)

In [26]:
postulaciones['timestamp'] = postulaciones['timestamp'].apply(lambda x: pendulum.parse(x))

In [27]:
postulaciones['month']     = postulaciones['timestamp'].apply(lambda x: x.month).astype(int) 
postulaciones['day']       = postulaciones['timestamp'].apply(lambda x: x.day).astype(int) 
postulaciones['dayofweek'] = postulaciones['timestamp'].apply(lambda x: x.dayofweek).astype(int) 
postulaciones['hour']      = postulaciones['timestamp'].apply(lambda x: x.hour).astype(int) 

In [28]:
postulaciones.drop('timestamp', axis=1, inplace=True)

## Proceso Avisos Online

In [29]:
avisos_online = pd.concat([
    pd.read_csv('src/fiuba_5_avisos_online.csv'),
    pd.read_csv('src/fiuba_5_avisos_online_2.csv')], axis=0)

In [30]:
avisos_online.drop_duplicates(inplace=True)

## Proceso el detalle de los avisos

In [31]:
avisos_detalles = pd.concat([
    pd.read_csv('src/fiuba_6_avisos_detalle.csv'),
    pd.read_csv('src/fiuba_6_avisos_detalle_2.csv'),
    pd.read_csv('src/fiuba_6_avisos_detalle_3.csv'),
    pd.read_csv('src/fiuba_6_avisos_detalle_4.csv')], axis=0)

In [32]:
avisos_detalles = avisos_detalles.drop('idpais', axis=1)
avisos_detalles = avisos_detalles.drop_duplicates('idaviso')

In [33]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Bioquímica','nombre_area'] = 'Bioquimica'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Química','nombre_area'] = 'Quimica'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Trabajo social','nombre_area'] = 'Trabajo Social'

In [34]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


## Borro el titulo, descripcion, mapacalle y ciudad

In [35]:
avisos_detalles = avisos_detalles.drop(['titulo','descripcion','denominacion_empresa','mapacalle','ciudad'],axis=1)

## Realizo un mapeo categorico hacia numerico

In [58]:
avisos_detalles['nombre_zona']     = avisos_detalles['nombre_zona']    .astype("category").cat.codes
avisos_detalles['tipo_de_trabajo'] = avisos_detalles['tipo_de_trabajo'].astype("category").cat.codes
avisos_detalles['nivel_laboral']   = avisos_detalles['nivel_laboral']  .astype("category").cat.codes
avisos_detalles['nombre_area']     = avisos_detalles['nombre_area']    .astype("category").cat.codes

In [37]:
avisos_detalles.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
0,1111556097,7,1,4,50
1,1111949392,7,1,4,89
2,1112145935,7,1,4,85
3,1112146010,7,1,4,144
4,1112211475,7,1,4,29


## Agrego informacion del postulante a la postulacion

In [38]:
postulaciones = pd.merge(postulaciones, postulantes, how='inner', on='idpostulante')

## Agrego la informacion del postulante a la vista

In [39]:
vistas = pd.merge(vistas, postulantes, how='inner', on='idpostulante')

## Columna "se_postulo"

### Si es una vista todabia no se postulo...

In [40]:
vistas['se_postulo'] = 0

### Si es una postulacion ya se postulo...

In [42]:
postulaciones['se_postulo'] = 1

### Si hay mas de una vista de una misma persona en un mismo aviso lo borro dejando solo una

In [43]:
vistas = vistas.drop_duplicates(subset=['idaviso','idpostulante'], keep='first')

## Hago el concat de las Vistas con las Postulaciones

In [44]:
vistas_postulaciones = pd.concat([postulaciones, vistas], axis=0)

## Agrego la informacion del aviso a vistas_postulaciones

In [45]:
vistas_postulaciones = pd.merge(vistas_postulaciones, avisos_detalles, how='inner', on='idaviso')

### Si hay una vista y una postulacion de una misma persona en un mismo aviso dejo solo la postulacion

In [46]:
vistas_postulaciones = vistas_postulaciones.drop_duplicates(subset=['idaviso','idpostulante'], keep='first')

In [47]:
vistas_postulaciones = vistas_postulaciones.fillna(-1)

In [48]:
vistas_postulaciones['valor_educacion'] = vistas_postulaciones['valor_educacion'].astype(int)

In [49]:
vistas_postulaciones.head()

,idaviso,idpostulante,month,day,dayofweek,hour,valor_educacion,sexo,edad,se_postulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
0,1112248724,NjlD,1,19,4,7,12,1,37,1,7,1,4,119
1,1112248724,MV6eM8N,1,30,1,23,11,-1,26,1,7,1,4,119
2,1112248724,5LzQkN,1,15,0,7,12,1,32,1,7,1,4,119
3,1112248724,8M2rOBz,1,30,1,22,12,1,26,1,7,1,4,119
4,1112248724,Dr6V3Da,1,23,1,15,12,-1,27,1,7,1,4,119


## Balanceo los -1

In [50]:
vistas_postulaciones['sexo'] = vistas_postulaciones['sexo'].apply(lambda x: np.random.choice([0, 1]) if(x==-1) else x)

In [51]:
vistas_postulaciones['valor_educacion'] = vistas_postulaciones['valor_educacion'].apply(lambda x: np.random.choice([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]) if(x==-1) else x)

## Hacer algo con esta online

In [52]:
### 

## Exporto finalmente el .CSV

In [53]:
vistas_postulaciones.to_csv('src.csv', encoding='utf-8', index=False)

In [54]:
vistas_postulaciones.shape

(9940108, 14)

In [55]:
vistas_postulaciones.groupby('se_postulo')['se_postulo'].count()

se_postulo
0    3752179
1    6187929
Name: se_postulo, dtype: int64